# Guided Project - Predicting the Stock Market
## Project Introduction
We are going to work witn SP500 index data and use clustering to predict stock market prices.

### Import Libraries & Data

In [108]:
# import libraries
import pandas as pd
from datetime import datetime
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

In [73]:
# import data
df = pd.read_csv('sphist.csv')

In [74]:
df.describe()

,Open,High,Low,Close,Volume,Adj Close
count,16590.000000,16590.000000,16590.000000,16590.000000,1.659000e+04,16590.000000
mean,482.570941,485.624237,479.367501,482.692491,7.940099e+08,482.692491
std,554.889186,558.186049,551.367625,555.007904,1.456582e+09,555.007904
min,16.660000,16.660000,16.660000,16.660000,6.800000e+05,16.660000
25%,83.860001,84.594997,83.139997,83.860001,7.610000e+06,83.860001
50%,144.049996,145.294998,143.105004,144.264999,7.170500e+07,144.264999
75%,950.722488,956.665024,941.969986,950.797500,7.866750e+08,950.797500
max,2130.360107,2134.719971,2126.060059,2130.820068,1.145623e+10,2130.820068


In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16590 entries, 0 to 16589
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       16590 non-null  object 
 1   Open       16590 non-null  float64
 2   High       16590 non-null  float64
 3   Low        16590 non-null  float64
 4   Close      16590 non-null  float64
 5   Volume     16590 non-null  float64
 6   Adj Close  16590 non-null  float64
dtypes: float64(6), object(1)
memory usage: 907.4+ KB


In [76]:
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(by='Date', ascending=True)

### Generate Indicators
We are going to use trailing close prices for 5, 15 and 30 days.

In [77]:
# set up the rolling averages
df['close_rolling_5_mean'] = df['Close'].rolling(window=5).apply(np.mean).shift(1)
df['close_rolling_15_mean'] = df['Close'].rolling(window=15).apply(np.mean).shift(1)
df['close_rolling_30_mean'] = df['Close'].rolling(window=30).apply(np.mean).shift(1)
df['close_rolling_365_mean'] = df['Close'].rolling(window=365).apply(np.mean).shift(1)

In [78]:
# check the shift worked fine
df[['Close','close_rolling_5_mean','close_rolling_15_mean','close_rolling_30_mean','close_rolling_365_mean']].head(35)

,Close,close_rolling_5_mean,close_rolling_15_mean,close_rolling_30_mean,close_rolling_365_mean
16589,16.660000,NaN,NaN,NaN,NaN
16588,16.850000,NaN,NaN,NaN,NaN
16587,16.930000,NaN,NaN,NaN,NaN
16586,16.980000,NaN,NaN,NaN,NaN
16585,17.080000,NaN,NaN,NaN,NaN
16584,17.030001,16.900,NaN,NaN,NaN
16583,17.090000,16.974,NaN,NaN,NaN
16582,16.760000,17.022,NaN,NaN,NaN
16581,16.670000,16.988,NaN,NaN,NaN
16580,16.719999,16.926,NaN,NaN,NaN


Now that we see our code works let's create more indicators.

In [79]:
# set up more indicators
df['close_rolling_5_var'] = df['Close'].rolling(window=5).apply(np.var).shift(1)
df['close_rolling_15_var'] = df['Close'].rolling(window=15).apply(np.var).shift(1)
df['close_rolling_30_var'] = df['Close'].rolling(window=30).apply(np.var).shift(1)
df['close_rolling_365_var'] = df['Close'].rolling(window=365).apply(np.var).shift(1)

In [86]:
# check the var indicators
df[['Date','Close','close_rolling_5_var','close_rolling_15_var','close_rolling_30_var','close_rolling_365_var']].head(35)

,Date,Close,close_rolling_5_var,close_rolling_15_var,close_rolling_30_var,close_rolling_365_var
16224,1951-06-19,22.020000,0.052520,0.059545,0.216816,3.196227
16223,1951-06-20,21.910000,0.036520,0.063505,0.191121,3.192847
16222,1951-06-21,21.780001,0.006856,0.063953,0.163650,3.190370
16221,1951-06-22,21.549999,0.010600,0.063907,0.131276,3.186971
16220,1951-06-25,21.290001,0.033336,0.062532,0.104716,3.181191
16219,1951-06-26,21.299999,0.068600,0.059856,0.088885,3.173743
16218,1951-06-27,21.370001,0.062264,0.061264,0.077718,3.165449
16217,1951-06-28,21.100000,0.034616,0.064745,0.076769,3.158386
16216,1951-06-29,20.959999,0.021016,0.083963,0.081378,3.143809
16215,1951-07-02,21.100000,0.022904,0.111193,0.084640,3.126418


In [87]:
# add ratios between indicators
df['mean_var_ration_rolling_5'] = df['close_rolling_5_mean'] / df['close_rolling_5_var']
df['mean_var_ration_rolling_15'] = df['close_rolling_15_mean'] / df['close_rolling_15_var']
df['mean_var_ration_rolling_30'] = df['close_rolling_30_mean'] / df['close_rolling_30_var']
df['mean_var_ration_rolling_365'] = df['close_rolling_365_mean'] / df['close_rolling_365_var']

In [100]:
# add indicators on volume, min max prices per rolling 365
df['volume_rolling_5_var'] = df['Volume'].rolling(window=5).apply(np.var).shift(1)
df['volume_rolling_365_var'] = df['Volume'].rolling(window=365).apply(np.var).shift(1)
df['close_rolling_365_min'] = df['Min'].rolling(window=365).apply(np.mean).shift(1)
df['close_rolling_365_max'] = df['Max'].rolling(window=365).apply(np.mean).shift(1)

KeyError: 'Min'

In [99]:
# create function for indicators:
def create_indicator(df, col, name, window, function):
    df[name] = df[col].rolling(window=window).apply().shift(1)

### Data Cleaning
We are going to remove NaN records and dates before 1950-01-03 to accomodate 365 rolling metrics

In [101]:
# remove records before '1950-01-03' as they would have NaN for rolling 365 metrics
df = df[df['Date'] > datetime(year=1951, month=1, day=2)]
# drop NaN values
df = df.dropna(axis=0)
df.head()

,Date,Open,High,Low,Close,Volume,Adj Close,close_rolling_5_mean,close_rolling_15_mean,close_rolling_30_mean,...,close_rolling_30_var,close_rolling_365_var,mean_var_ration_rolling_5,mean_var_ration_rolling_15,mean_var_ration_rolling_30,mean_var_ration_rolling_365,volume_rolling_5_var,volume_rolling_365_var,close_rolling_365_min,close_rolling_365_max
15494,1954-05-20,28.82,28.82,28.82,28.82,2070000.0,28.82,28.753999,28.532000,28.093667,...,0.240903,1.654401,2499.462573,543.923568,116.618065,15.304454,1.818400e+10,1.968465e+11,22.709999,28.85
15493,1954-05-21,28.99,28.99,28.99,28.99,2620000.0,28.99,28.806000,28.574667,28.150666,...,0.222986,1.687644,13435.403533,596.488595,126.243928,15.008179,9.760000e+09,1.975532e+11,22.709999,28.85
15492,1954-05-24,29.00,29.00,29.00,29.00,2330000.0,29.00,28.844000,28.623333,28.204333,...,0.223791,1.723787,3864.390156,567.523769,126.029642,14.698637,4.356000e+10,2.004795e+11,22.709999,28.99
15491,1954-05-25,28.93,28.93,28.93,28.93,2050000.0,28.93,28.876000,28.676000,28.258333,...,0.219327,1.759508,2554.479866,627.064509,128.840973,14.405010,3.497600e+10,2.012412e+11,22.709999,29.00
15490,1954-05-26,29.17,29.17,29.17,29.17,2180000.0,29.17,28.892000,28.719333,28.303667,...,0.216517,1.793537,2513.206932,761.796223,130.722865,14.136202,4.441600e+10,2.016152e+11,22.709999,29.00


## Regression Models
Now that we have cleaned the dataset we can split into train and test:
1. Train dates before '2013-01-01'
2. Test dates after or equal to '2013-01-01'

Then we can train a model and test results using MSE and RMSE.
We are going to try:
1. Linear Regression Model

In [102]:
# split dataset in train and test
train_df = df[df['Date'] < '2013-01-01'].copy()
test_df = df[df['Date'] >= '2013-01-01'].copy()

In [103]:
df.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close',
       'close_rolling_5_mean', 'close_rolling_15_mean',
       'close_rolling_30_mean', 'close_rolling_365_mean',
       'close_rolling_5_var', 'close_rolling_15_var', 'close_rolling_30_var',
       'close_rolling_365_var', 'mean_var_ration_rolling_5',
       'mean_var_ration_rolling_15', 'mean_var_ration_rolling_30',
       'mean_var_ration_rolling_365', 'volume_rolling_5_var',
       'volume_rolling_365_var', 'close_rolling_365_min',
       'close_rolling_365_max'],
      dtype='object')

In [104]:
# select features and target
features_base = ['close_rolling_5_mean','close_rolling_365_mean','close_rolling_5_var',
                 'close_rolling_365_var', 'mean_var_ration_rolling_5',
                 'mean_var_ration_rolling_365']
features_full = ['close_rolling_5_mean','close_rolling_15_mean','close_rolling_30_mean','close_rolling_365_mean','close_rolling_5_var','close_rolling_15_var','close_rolling_30_var',
            'close_rolling_365_var', 'mean_var_ration_rolling_5',
            'mean_var_ration_rolling_15', 'mean_var_ration_rolling_30',
            'mean_var_ration_rolling_365']
features_new =  ['close_rolling_5_mean','close_rolling_365_mean','close_rolling_5_var',
                 'close_rolling_365_var', 'mean_var_ration_rolling_5',
                 'mean_var_ration_rolling_365', 'volume_rolling_5_var',
                 'volume_rolling_365_var']
target = 'Close'

### Linear Model

In [105]:
# initiate model and train and test with initial set of features
lr = LinearRegression()
lr.fit(train_df[features_base], train_df[target])
predictions = lr.predict(test_df[features_base])
mse = mean_squared_error(test_df[target], predictions)
rmse = np.sqrt(mse)
print('MSE base features:', mse)
print('RMSE base features:', rmse)

MSE base features: 483.2006446429048
RMSE base features: 21.98182532554803


We get an MSE of 483 and RMSE of 22.

In [106]:
# train and test with more features
lr = LinearRegression()
lr.fit(train_df[features_full], train_df[target])
predictions = lr.predict(test_df[features_full])
mse = mean_squared_error(test_df[target], predictions)
rmse = np.sqrt(mse)
print('MSE full features:', mse)
print('RMSE full features:', rmse)

MSE full features: 477.58196890731705
RMSE full features: 21.853648869406616


Adding more of the same kind of indicators to the features list does not improve the model quality. Let's try with different indicators.

In [107]:
# train and test model with features from volume as well
lr = LinearRegression()
lr.fit(train_df[features_new], train_df[target])
predictions = lr.predict(test_df[features_new])
mse = mean_squared_error(test_df[target], predictions)
rmse = np.sqrt(mse)
print('MSE full features:', mse)
print('RMSE full features:', rmse)

MSE full features: 2942.2159524988656
RMSE full features: 54.24219715773749


Adding volume indicators to the model decreased its precision in the train & test environment.

## Next Steps:
Take a different approach in feature engineering and building new indicators as well as testing different models (Random Forest).